In [1]:
import os 
import pandas as pd
import pickle
import re
from operator import itemgetter
import random
import seaborn as sns
import matplotlib.ticker as ticker
from matplotlib import pyplot as plt
%matplotlib inline

# Define os parametros necessarios para o teste

class Parametros:
    def __init__(self, seed, length, a, b, n_calls, band, file, modulacao):
        self.seed = seed
        self.length = length
        self.a = a
        self.b = b
        self.n_calls = n_calls
        self.band = band
        self.file = file
        self.modulacao = modulacao
        
# Lê arquivo de data utilizado
        
    def gera_data(self):
        a_file = open(self.file, "rb")
        return pickle.load(a_file)
        
# Gera o vetor de SNR
        
    def gera_vetor_snr(self):
        random.seed(self.seed)
        sinalRuido = []
        for i in range(0, self.length):
            sinalRuido.append(round(random.uniform(self.a, self.b), 1))
        return sinalRuido
    
# Calcula a banda requerida para cada chamada
    
    def calcula_banda_requerida(self, rate):
        codecRate = 64 # Kbps
        codecPayload = 160  # Bytes
        pps = 50 # milisegundos
        cabecalhoIP = 40 # Bytes 
        return ((((codecPayload * rate + cabecalhoIP) * 8) * pps) / 1000) * self.n_calls
    
# Busca o resultado no dataset baseado nos parametros de teste 
    
    def busca_resultados(self, snr):
        availableConfigs = []
        bandwidth_results = []

        for mod, param in self.gera_data().items():
            if mod == self.modulacao:
                if snr in param.keys():
                    for tupla in param.get(snr):
                        rate = tupla[3]
                        if self.calcula_banda_requerida(rate) <= self.band:
                            availableConfigs.append(tupla)
        return availableConfigs
        
# Gera os gráficos de resultados de teste
        
    def gera_graficos():
        pass
    
    
class Teste(Parametros):
#     def __init__(self, modulacao, band):
#         super().__init__(modulacao, band)


    def modelo_adaptativo(self):
#         Necessita corrigir!!!
        results = []
        for snr in self.gera_vetor_snr():
            value = sorted(sorted(self.busca_resultados(snr), key=lambda element: element[3]), 
                key=lambda element: element[0], reverse=True)[0]
            results.append(value)
        return results
            

    def modelo_fixo(self, **kwargs):
        results = []
        if kwargs.get('code') == False:
            pattern = 'No Code'
        else:
            try:
                n = kwargs['n']
                k = kwargs['k']
            except:
                return []
            pattern = 'RS('+ str(n) + ', ' + str(k) + ')'
        for snr in self.gera_vetor_snr():
            for element in self.busca_resultados(snr):
                if element[2] == pattern:
                    results.append(element)
        return results
        
        
    def compara(self, **kwargs):
        test1 = self.modelo_adaptativo()
        test2 = self.modelo_fixo(**kwargs)
        return test1, test2
        
        

In [2]:
# Args: | Seed | Taanho Vetor SNR | Valor inicial | Valor final | Arquivo de dados | Modulação |
resultado = Teste(0, 10, 3, 12, 1, 100, "data.pkl", 'BPSK')

In [3]:
resultado.modelo_adaptativo()

[(4.5, 0.0, 'RS(255, 247)', 1.0323886639676114),
 (4.5, 0.0, 'RS(255, 247)', 1.0323886639676114),
 (4.5, 0.0, 'RS(255, 225)', 1.1333333333333333),
 (4.5, 0.0, 'RS(255, 205)', 1.2439024390243902),
 (4.5, 0.0, 'RS(255, 225)', 1.1333333333333333),
 (4.5, 0.0, 'RS(255, 225)', 1.1333333333333333),
 (4.5, 0.0, 'RS(255, 247)', 1.0323886639676114),
 (4.5, 0.0, 'RS(240, 200)', 1.2),
 (4.5, 0.0, 'RS(255, 225)', 1.1333333333333333),
 (4.5, 0.0, 'RS(255, 247)', 1.0323886639676114)]

In [4]:
resultado.modelo_fixo(code=False)

[(4.3394, 8.056799999999999e-07, 'No Code', 1.0),
 (3.9939, 5.8536e-06, 'No Code', 1.0),
 (1.1915, 0.00098389, 'No Code', 1.0),
 (1.0823, 0.004616899999999999, 'No Code', 1.0),
 (1.5515, 0.00034796, 'No Code', 1.0),
 (1.1616, 0.0012472000000000002, 'No Code', 1.0),
 (4.0387, 3.1643e-06, 'No Code', 1.0),
 (1.0847, 0.0032052, 'No Code', 1.0),
 (1.3677, 0.00052493, 'No Code', 1.0),
 (2.1776, 0.0001183, 'No Code', 1.0)]

In [5]:
resultado.gera_vetor_snr()

[10.6, 9.8, 6.8, 5.3, 7.6, 6.6, 10.1, 5.7, 7.3, 8.3]

In [6]:
lista_de_tuplas = resultado.busca_resultados(10.9)

In [9]:
get_best(lista_de_tuplas)

[(4.5, 0.0, 'RS(255, 247)', 1.0323886639676114),
 (4.5, 0.0, 'RS(255, 225)', 1.1333333333333333),
 (4.5, 0.0, 'RS(240, 200)', 1.2),
 (4.5, 0.0, 'RS(255, 205)', 1.2439024390243902),
 (4.5, 0.0, 'RS(360, 280)', 1.2857142857142858),
 (4.4597, 2.8541e-07, 'No Code', 1.0)]

In [8]:
def get_maior_mos(tupla):
    return sorted(tupla, key=lambda element: element[0], reverse=True)

def get_menor_rate(tupla):
    return sorted(tupla, key=lambda element: element[3])

def get_best(tupla):
    return sorted(sorted(tupla, key=lambda element: element[3]),
                  key=lambda element: element[0], reverse=True)

In [10]:
def otimiza(lista_de_tuplas):
    temp = []
    
    # Se maior resultado de MOS estiver menor do que 2.58, 
    # retorna configuração com menor rate [prioridade largura de banda]
    
    if get_maior_mos(lista_de_tuplas)[0][0] < 2.58:
        result_tupla = get_menor_rate(lista_de_tuplas)[0]
        
    # Se maior resultado de MOS estiver maior do que 4.03,
    # seleciona todas configurações com valor acima de 4.03 e seleciona com menor rate [prioridade largura de banda]     
        
    elif get_maior_mos(lista_de_tuplas)[0][0] > 4.03:
        for tupla in lista_de_tuplas:
            if tupla[0] > 4.03:
                temp.append(tupla)
        result_tupla = get_menor_rate(temp)[0]
        
    # Caso contrario é escolhido o maior resultado de MOS com o menor custo de banda        
        
    else:
        result_tupla = get_best(lista_de_tuplas)[0]
    return result_tupla
        

In [11]:
otimiza(lista_de_tuplas)

(4.4597, 2.8541e-07, 'No Code', 1.0)